In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, GRU, Flatten, Dropout
import time

In [14]:
df_X1 = pd.read_pickle(r"C:\Users\Hong\Downloads\MediaPipe_Testing\MediaPipePoseEstimation\Deep_Learning\X_array_2.pkl")
df_X2 = pd.read_pickle(r"C:\Users\Hong\Downloads\MediaPipe_Testing\MediaPipePoseEstimation\Deep_Learning\Y_array_2.pkl")
df_X3 = pd.read_pickle(r"C:\Users\Hong\Downloads\MediaPipe_Testing\MediaPipePoseEstimation\Deep_Learning\Z_array_2.pkl")
df_Y = pd.read_pickle(r"C:\Users\Hong\Downloads\MediaPipe_Testing\MediaPipePoseEstimation\Deep_Learning\Group_2.pkl")

In [15]:
# Convert the lists to numpy arrays for compatibility with Keras
X1 = np.array(df_X1)
X2 = np.array(df_X2)
X3 = np.array(df_X3)
Y = np.array(df_Y)

In [16]:
X = np.concatenate((X1, X2, X3), axis=1)

In [17]:
print(X.shape)

(40, 99, 847)


In [18]:
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [19]:
print(X_train.shape)

(32, 99, 847)


In [20]:
print(X_test.shape)

(8, 99, 847)


In [21]:
model = Sequential()
model.add(GRU(100, activation="tanh", return_sequences=True, input_shape=(99, 847)))
model.add(Dropout(0.15))
model.add(GRU(80, activation="tanh", return_sequences=True))
model.add(Dropout(0.15))
model.add(GRU(50, activation="tanh", return_sequences=False))
model.add(Dropout(0.15))
model.add(Dense(3, activation='sigmoid'))  # Change GRU to Dense layer
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_3 (GRU)                 (None, 99, 100)           284700    
                                                                 
 dropout_3 (Dropout)         (None, 99, 100)           0         
                                                                 
 gru_4 (GRU)                 (None, 99, 80)            43680     
                                                                 
 dropout_4 (Dropout)         (None, 99, 80)            0         
                                                                 
 gru_5 (GRU)                 (None, 50)                19800     
                                                                 
 dropout_5 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                

In [22]:
#model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [23]:
start_time = time.time()  # Start the timer

In [24]:
# Train the model
model.fit(X_train, Y_train, batch_size=1, epochs=1000)

Epoch 1/30000
32/32 [==============================] - 9s 42ms/step - loss: 0.6556 - accuracy: 0.6562
Epoch 2/30000
32/32 [==============================] - 1s 42ms/step - loss: 0.4903 - accuracy: 0.8125
Epoch 3/30000
32/32 [==============================] - 4s 139ms/step - loss: 0.4315 - accuracy: 0.8125
Epoch 4/30000
32/32 [==============================] - 4s 118ms/step - loss: 0.3035 - accuracy: 0.8438
Epoch 5/30000
32/32 [==============================] - 4s 114ms/step - loss: 0.3576 - accuracy: 0.7812
Epoch 6/30000
32/32 [==============================] - 3s 104ms/step - loss: 0.2421 - accuracy: 0.8438
Epoch 7/30000
32/32 [==============================] - 6s 173ms/step - loss: 0.2464 - accuracy: 0.8750
Epoch 8/30000
32/32 [==============================] - 5s 148ms/step - loss: 0.1995 - accuracy: 0.9062
Epoch 9/30000
32/32 [==============================] - 5s 157ms/step - loss: 0.2547 - accuracy: 0.8750
Epoch 10/30000
32/32 [==============================] - 5s 144ms/step - los

KeyboardInterrupt: 

In [39]:
end_time = time.time()  # End the timer

In [40]:
print(f"The processing time of the GRU is {end_time - start_time} seconds.")  # Print the processing time

The processing time of the GRU is 9812.131520748138 seconds.


In [41]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, Y_test,batch_size=2)

4/4 [==============================] - 1s 5ms/step - loss: 9.4539 - accuracy: 0.6250


In [42]:
# Print the test loss and test accuracy
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Test Loss: 9.453935623168945
Test Accuracy: 0.625


In [43]:
# Make predictions on the test set
Y_pred = model.predict(X_test)

1/1 [==============================] - 0s 475ms/step


In [44]:
# Convert predictions to class labels
Y_pred_classes = np.argmax(Y_pred, axis=1)

In [45]:
from sklearn.metrics import classification_report

In [46]:
print(classification_report(Y_test, Y_pred_classes))

              precision    recall  f1-score   support

           0       1.00      0.83      0.91         6
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1

    accuracy                           0.62         8
   macro avg       0.33      0.28      0.30         8
weighted avg       0.75      0.62      0.68         8

